<a href="https://colab.research.google.com/github/USCbiostats/PM520/blob/main/Lab_6_ExpFam_Divergences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# It's a Family Affair, or: Exponential Families, Natural Gradient Descent, & Statistical Divergences